In [7]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [9]:
subject = 'São Paulo - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [10]:
data = pd.read_csv('2003_01_model_input_SP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland,São Paulo - Consumo de Cimento (t)
0,2003,5232.682934,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,12.350925,...,84.902756,1.930011,0.812723,1.034903e+09,4.730940e+07,23.895559,9.819129e+08,8.310335,3.922845,9528.051000
1,2004,5521.391778,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,12.479616,...,83.152712,2.213838,0.814104,1.049168e+09,4.755584e+07,23.966513,9.861577e+08,8.236641,4.120177,9148.318000
2,2005,6102.302057,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,14.614783,...,79.675706,2.489308,0.815359,1.062582e+09,4.779561e+07,24.046393,9.901728e+08,8.212526,4.124581,9724.722000
3,2006,6788.458005,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,18.279600,...,80.384734,2.171811,0.816488,1.074248e+09,4.799886e+07,24.099559,9.927879e+08,8.126466,3.981310,10863.790000
4,2007,7337.194102,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,19.035902,...,78.901058,1.896576,0.817495,1.084443e+09,4.833495e+07,24.140672,9.945643e+08,8.101259,3.847089,11775.453000
5,2008,7770.929330,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,18.097653,...,78.961939,1.773094,0.818381,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,8.071263,3.773615,13476.268000
6,2009,8316.504885,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,17.458446,...,80.559826,1.537503,0.819150,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8.100327,3.771810,13304.301000
7,2010,8835.509117,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,17.477114,...,81.499868,1.367891,0.819803,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8.064867,3.913291,14534.980000
8,2011,9192.769768,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,17.341677,...,82.321001,1.335640,0.820344,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,8.053175,4.060906,14871.943000
9,2012,9435.198835,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,16.745541,...,82.512436,1.341854,0.820550,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,10.058484,4.219677,15159.683000


In [11]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
0,5232.682934,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,12.350925,835.133886,...,12.164126,84.902756,1.930011,0.812723,1.034903e+09,4.730940e+07,23.895559,9.819129e+08,8.310335,3.922845
1,5521.391778,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,12.479616,834.672463,...,12.263090,83.152712,2.213838,0.814104,1.049168e+09,4.755584e+07,23.966513,9.861577e+08,8.236641,4.120177
2,6102.302057,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,14.614783,834.963398,...,11.702282,79.675706,2.489308,0.815359,1.062582e+09,4.779561e+07,24.046393,9.901728e+08,8.212526,4.124581
3,6788.458005,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,18.279600,835.110366,...,11.378895,80.384734,2.171811,0.816488,1.074248e+09,4.799886e+07,24.099559,9.927879e+08,8.126466,3.981310
4,7337.194102,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,19.035902,835.834715,...,10.870297,78.901058,1.896576,0.817495,1.084443e+09,4.833495e+07,24.140672,9.945643e+08,8.101259,3.847089
5,7770.929330,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,18.097653,836.334300,...,11.136027,78.961939,1.773094,0.818381,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,8.071263,3.773615
6,8316.504885,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,17.458446,836.629901,...,11.417327,80.559826,1.537503,0.819150,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8.100327,3.771810
7,8835.509117,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,17.477114,836.656415,...,11.288684,81.499868,1.367891,0.819803,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8.064867,3.913291
8,9192.769768,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,17.341677,836.557224,...,11.391381,82.321001,1.335640,0.820344,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,8.053175,4.060906
9,9435.198835,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,16.745541,836.685833,...,11.497643,82.512436,1.341854,0.820550,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,10.058484,4.219677


In [12]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      9148.318000
1      9724.722000
2     10863.790000
3     11775.453000
4     13476.268000
5     13304.301000
6     14534.980000
7     14871.943000
8     15159.683000
9     15089.297000
10    13714.511286
11    11767.653429
12     9811.084286
13     9918.501000
14    10001.077000
15    10458.276000
16    11181.809000
17    12113.838000
18    12383.649000
19             NaN
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [13]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
0,-1.373010,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-2.125950,-1.466315,...,1.039431,1.852886,0.743660,-2.115587,-1.653768,-0.358309,-1.337901,-2.477902,-0.810130,-0.887599
1,-1.170521,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-2.054857,-2.039253,...,1.271506,0.792270,1.573384,-1.679161,-1.427561,-0.071270,-0.606200,-1.773047,-0.856132,-0.454655
2,-0.763092,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.875330,-1.678006,...,-0.043622,-1.314973,2.378677,-1.282958,-1.214848,0.208010,0.217547,-1.106338,-0.871185,-0.444993
3,-0.281848,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,1.149220,-1.495519,...,-0.801982,-0.885266,1.450524,-0.926225,-1.029862,0.444753,0.765810,-0.672087,-0.924906,-0.759327
4,0.103015,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,1.567023,-0.596113,...,-1.994674,-1.784449,0.645919,-0.608210,-0.868197,0.836216,1.189782,-0.377108,-0.940640,-1.053806
5,0.407220,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,1.048707,0.024209,...,-1.371522,-1.747552,0.284936,-0.328158,-0.742092,1.064358,1.221901,-0.357446,-0.959365,-1.215007
6,0.789867,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,0.695591,0.391249,...,-0.711856,-0.779151,-0.403776,-0.085317,-0.642613,1.300921,1.198844,-0.352980,-0.941222,-1.218968
7,1.153877,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,0.705903,0.424171,...,-1.013531,-0.209438,-0.899609,0.121064,-0.240784,1.432729,1.250915,0.280020,-0.963357,-0.908559
8,1.404446,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.631084,0.301008,...,-0.772701,0.288211,-0.993890,0.291741,0.118022,1.082972,1.123993,0.651955,-0.970656,-0.584694
9,1.574477,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.301762,0.460699,...,-0.523512,0.404230,-0.975724,0.356913,0.436508,0.725426,0.663733,0.888800,0.281103,-0.236354


In [14]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      9148.318000
1      9724.722000
2     10863.790000
3     11775.453000
4     13476.268000
5     13304.301000
6     14534.980000
7     14871.943000
8     15159.683000
9     15089.297000
10    13714.511286
11    11767.653429
12     9811.084286
13     9918.501000
14    10001.077000
15    10458.276000
16    11181.809000
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
17,-0.733293,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-0.517117,0.377546,...,-0.416421,-0.692233,-1.206794,-0.064924,0.776026,-1.154968,-1.420169,0.021261,0.852494,2.586291


In [16]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    12113.838
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [17]:
train_input.iloc[-(len(train_input)//5):]

,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
14,-1.049614,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,0.433674,0.859425,...,0.666160,0.548843,-0.506442,1.243424,1.189596,-1.504471,-1.181425,0.679759,1.172636,1.148630
15,-1.149965,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,0.227085,0.838221,...,1.260579,1.250121,0.027458,1.023912,1.150795,-1.545321,-1.270370,0.550558,1.068127,1.412478
16,-0.995655,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,-0.833350,0.733064,...,0.976765,0.923485,-0.574513,0.612137,1.039649,-1.465420,-1.334622,0.398702,0.992960,1.960645


In [18]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [21]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[4031584217, 1343273284, 3042199330, 1217026960, 1873525885, 3461594054, 1535989307, 1489147984, 1859730151, 2279258393, 2366420688, 3693179251, 851271110, 3170389419, 1906294117, 4050066332, 353417237, 1293673945, 3529908768, 2792407140, 543343175, 2070318139, 2302179271, 2439458802, 914819434, 4018268500, 4177481018, 775033522, 4169351595, 2927060570, 177918552, 635009367, 1538560047, 3680163202, 4191559338, 426396290, 802086877, 3975794083, 3779999434, 298833733, 2228302168, 898660604, 4015998077, 2755194225, 4075508802, 1800995272, 3402286717, 3960794937, 1399100473, 1551590091, 806387081, 148664608, 2940403928, 1424965481, 398330743, 1966699232, 4214923788, 4036058018, 3972683542, 109307334, 231134231, 4177836694, 1206690753, 3028421213, 2247218951, 3153727068, 1373996753, 2495882661, 373313803, 2527016717, 4100329192, 2025189840, 4146448552, 884904842, 1669316659, 1766499362, 4196851216, 2125705573, 417849824, 77055799, 2179592336, 2736229428, 2984828597, 93712855, 2827859156, 52

2023-10-05 14:06:11.919977: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-05 14:06:11.939307: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 221451.078125


Step: 24 ___________________________________________
val_loss: 73310.40625


Step: 25 ___________________________________________
val_loss: 72762.1015625


Step: 26 ___________________________________________
val_loss: 169058.359375


Step: 27 ___________________________________________
val_loss: 6406.78271484375


Step: 28 ___________________________________________
val_loss: 240504.609375


Step: 29 ___________________________________________
val_loss: 18043.13671875


Step: 30 ___________________________________________
val_loss: 70390.9453125


Step: 31 ___________________________________________
val_loss: 153084.484375


Step: 32 ___________________________________________
val_loss: 218472.671875


Step: 33 ___________________________________________
val_loss: 236157.8125


Step: 34 ___________________________________________
val_loss: 5863.27197265625


Step: 35 ___________________________________________
val_loss: 16586.978515625


Step: 36 ____________

2023-10-05 14:25:16.862296: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-05 14:25:16.894191: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-05 14:25:41.140026: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-05 14:25:41.174150: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 119433.78125


Step: 89 ___________________________________________
val_loss: 26153.259765625


Step: 90 ___________________________________________
val_loss: 159755.140625


Step: 91 ___________________________________________
val_loss: 86605.7890625


Step: 92 ___________________________________________
val_loss: 236643.125


Step: 93 ___________________________________________
val_loss: 151968.265625


Step: 94 ___________________________________________
val_loss: 123391.21875


Step: 95 ___________________________________________
val_loss: 226630.171875


Step: 96 ___________________________________________
val_loss: 141805.4375


Step: 97 ___________________________________________
val_loss: 92624.25


Step: 98 ___________________________________________
val_loss: 97828.40625


Step: 99 ___________________________________________
val_loss: 80716.1484375


final_seed: 2279258393


In [22]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 454ms/step - loss: 149361264.0000 - val_loss: 110880264.0000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 147490992.0000 - val_loss: 203811008.0000
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 148726576.0000 - val_loss: 131562008.0000
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 134382160.0000 - val_loss: 102584264.0000
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 119588808.0000 - val_loss: 57958644.0000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 92740672.0000 - val_loss: 23850128.0000
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 88131408.0000 - val_loss: 17916518.0000
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 104782184.0000 - val_loss: 49225408.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 107

1/1 [==============================] - 0s 13ms/step - loss: 41717592.0000 - val_loss: 28953510.0000
Epoch 73/10000
1/1 [==============================] - 0s 13ms/step - loss: 32485120.0000 - val_loss: 32132374.0000
Epoch 74/10000
1/1 [==============================] - 0s 13ms/step - loss: 47465992.0000 - val_loss: 20247182.0000
Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 50299824.0000 - val_loss: 14901299.0000
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 29422486.0000 - val_loss: 14628799.0000
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 26772230.0000 - val_loss: 17632512.0000
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 33343504.0000 - val_loss: 18604766.0000
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 30301934.0000 - val_loss: 10038748.0000
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 30279974.0000 - val_

1/1 [==============================] - 0s 13ms/step - loss: 11813938.0000 - val_loss: 9586967.0000
Epoch 144/10000
1/1 [==============================] - 0s 13ms/step - loss: 8814299.0000 - val_loss: 7490585.5000
Epoch 145/10000
1/1 [==============================] - 0s 13ms/step - loss: 10669062.0000 - val_loss: 7867608.0000
Epoch 146/10000
1/1 [==============================] - 0s 13ms/step - loss: 15004828.0000 - val_loss: 7895832.0000
Epoch 147/10000
1/1 [==============================] - 0s 13ms/step - loss: 9347680.0000 - val_loss: 6699856.0000
Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 10455808.0000 - val_loss: 5863536.5000
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 15762619.0000 - val_loss: 3855001.0000
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 9629174.0000 - val_loss: 6039689.5000
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 11503267.0000 - val_los

Epoch 215/10000
1/1 [==============================] - 0s 13ms/step - loss: 5262520.0000 - val_loss: 1514415.0000
Epoch 216/10000
1/1 [==============================] - 0s 13ms/step - loss: 9603036.0000 - val_loss: 755519.5000
Epoch 217/10000
1/1 [==============================] - 0s 13ms/step - loss: 4604088.5000 - val_loss: 1836987.0000
Epoch 218/10000
1/1 [==============================] - 0s 13ms/step - loss: 6400533.5000 - val_loss: 305827.7500
Epoch 219/10000
1/1 [==============================] - 0s 13ms/step - loss: 5650346.0000 - val_loss: 810222.1875
Epoch 220/10000
1/1 [==============================] - 0s 13ms/step - loss: 6269796.5000 - val_loss: 1678150.6250
Epoch 221/10000
1/1 [==============================] - 0s 13ms/step - loss: 5889402.0000 - val_loss: 1145325.8750
Epoch 222/10000
1/1 [==============================] - 0s 13ms/step - loss: 4634752.5000 - val_loss: 294437.9375
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 4444995.0000 - v

1/1 [==============================] - 0s 13ms/step - loss: 2720946.0000 - val_loss: 1681130.8750
Epoch 288/10000
1/1 [==============================] - 0s 13ms/step - loss: 4547221.5000 - val_loss: 677180.1875
Epoch 289/10000
1/1 [==============================] - 0s 13ms/step - loss: 3095133.7500 - val_loss: 1181208.3750
Epoch 290/10000
1/1 [==============================] - 0s 13ms/step - loss: 2323439.7500 - val_loss: 2164631.7500
Epoch 291/10000
1/1 [==============================] - 0s 13ms/step - loss: 1995041.3750 - val_loss: 2419237.2500
Epoch 292/10000
1/1 [==============================] - 0s 13ms/step - loss: 3671981.2500 - val_loss: 1117314.8750
Epoch 293/10000
1/1 [==============================] - 0s 13ms/step - loss: 2928782.2500 - val_loss: 1074076.7500
Epoch 294/10000
1/1 [==============================] - 0s 13ms/step - loss: 4748210.5000 - val_loss: 1026547.7500
Epoch 295/10000
1/1 [==============================] - 0s 13ms/step - loss: 2905450.0000 - val_loss: 2339

...

1/1 [==============================] - 0s 13ms/step - loss: 2373803.5000 - val_loss: 1756311.1250
Epoch 792/10000
1/1 [==============================] - 0s 13ms/step - loss: 2526434.0000 - val_loss: 1403143.8750
Epoch 793/10000
1/1 [==============================] - 0s 13ms/step - loss: 2647953.7500 - val_loss: 1856748.8750
Epoch 794/10000
1/1 [==============================] - 0s 13ms/step - loss: 2388065.5000 - val_loss: 1564350.3750
Epoch 795/10000
1/1 [==============================] - 0s 13ms/step - loss: 3441713.0000 - val_loss: 2453991.0000
Epoch 796/10000
1/1 [==============================] - 0s 13ms/step - loss: 3726400.0000 - val_loss: 3054020.0000
Epoch 797/10000
1/1 [==============================] - 0s 13ms/step - loss: 2017596.5000 - val_loss: 1482602.6250
Epoch 798/10000
1/1 [==============================] - 0s 13ms/step - loss: 3172307.0000 - val_loss: 1978728.5000
Epoch 799/10000
1/1 [==============================] - 0s 13ms/step - loss: 2999191.5000 - val_loss: 115

1/1 [==============================] - 0s 13ms/step - loss: 2774706.0000 - val_loss: 1171849.2500
Epoch 864/10000
1/1 [==============================] - 0s 13ms/step - loss: 2062853.6250 - val_loss: 767197.0625
Epoch 865/10000
1/1 [==============================] - 0s 13ms/step - loss: 2072886.8750 - val_loss: 620196.9375
Epoch 866/10000
1/1 [==============================] - 0s 13ms/step - loss: 3010152.0000 - val_loss: 844269.1875
Epoch 867/10000
1/1 [==============================] - 0s 13ms/step - loss: 2500625.5000 - val_loss: 364552.2812
Epoch 868/10000
1/1 [==============================] - 0s 13ms/step - loss: 2417545.5000 - val_loss: 898655.0625
Epoch 869/10000
1/1 [==============================] - 0s 13ms/step - loss: 3166723.5000 - val_loss: 1049892.1250
Epoch 870/10000
1/1 [==============================] - 0s 13ms/step - loss: 2927014.0000 - val_loss: 267312.9688
Epoch 871/10000
1/1 [==============================] - 0s 13ms/step - loss: 4022775.5000 - val_loss: 725995.18

1/1 [==============================] - 0s 13ms/step - loss: 2106132.7500 - val_loss: 1747991.0000
Epoch 936/10000
1/1 [==============================] - 0s 13ms/step - loss: 3044670.0000 - val_loss: 680676.4375
Epoch 937/10000
1/1 [==============================] - 0s 13ms/step - loss: 2239187.0000 - val_loss: 1067968.2500
Epoch 938/10000
1/1 [==============================] - 0s 13ms/step - loss: 1852318.0000 - val_loss: 633652.5000
Epoch 939/10000
1/1 [==============================] - 0s 14ms/step - loss: 3530525.2500 - val_loss: 518852.0938
Epoch 940/10000
1/1 [==============================] - 0s 13ms/step - loss: 2233845.2500 - val_loss: 382472.4062
Epoch 941/10000
1/1 [==============================] - 0s 13ms/step - loss: 2911931.0000 - val_loss: 1231857.7500
Epoch 942/10000
1/1 [==============================] - 0s 13ms/step - loss: 2673083.0000 - val_loss: 938065.9375
Epoch 943/10000
1/1 [==============================] - 0s 13ms/step - loss: 2600850.2500 - val_loss: 569802.0

1/1 [==============================] - 0s 13ms/step - loss: 3289956.0000 - val_loss: 1106233.8750
Epoch 1008/10000
1/1 [==============================] - 0s 13ms/step - loss: 3015869.2500 - val_loss: 1431536.3750
Epoch 1009/10000
1/1 [==============================] - 0s 13ms/step - loss: 2239646.0000 - val_loss: 1194062.8750
Epoch 1010/10000
1/1 [==============================] - 0s 13ms/step - loss: 2047482.1250 - val_loss: 1523677.6250
Epoch 1011/10000
1/1 [==============================] - 0s 13ms/step - loss: 2260144.0000 - val_loss: 1309747.6250
Epoch 1012/10000
1/1 [==============================] - 0s 13ms/step - loss: 3132302.0000 - val_loss: 897673.8125
Epoch 1013/10000
1/1 [==============================] - 0s 13ms/step - loss: 1754304.1250 - val_loss: 991598.0625
Epoch 1014/10000
1/1 [==============================] - 0s 13ms/step - loss: 1386561.1250 - val_loss: 1118189.3750
Epoch 1015/10000
1/1 [==============================] - 0s 13ms/step - loss: 2161340.7500 - val_los

Epoch 1079/10000
1/1 [==============================] - 0s 13ms/step - loss: 2609928.5000 - val_loss: 278053.8438
Epoch 1080/10000
1/1 [==============================] - 0s 13ms/step - loss: 3203454.5000 - val_loss: 415370.2500
Epoch 1081/10000
1/1 [==============================] - 0s 13ms/step - loss: 2616691.5000 - val_loss: 451078.0938
Epoch 1082/10000
1/1 [==============================] - 0s 13ms/step - loss: 2185186.0000 - val_loss: 702231.9375
Epoch 1083/10000
1/1 [==============================] - 0s 13ms/step - loss: 1830908.1250 - val_loss: 607723.9375
Epoch 1084/10000
1/1 [==============================] - 0s 13ms/step - loss: 1936359.3750 - val_loss: 973291.1875
Epoch 1085/10000
1/1 [==============================] - 0s 13ms/step - loss: 2452727.2500 - val_loss: 550275.3750
Epoch 1086/10000
1/1 [==============================] - 0s 13ms/step - loss: 2618011.5000 - val_loss: 1224105.8750
Epoch 1087/10000
1/1 [==============================] - 0s 13ms/step - loss: 2300502.00

In [23]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[12113.838](test_target) - [[10326.173]](prediction) = 1787.6651484374997


In [24]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [25]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
0,-1.435135,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.543973,-0.714086,...,1.400182,1.980130,0.021817,-1.700607,-1.593377,-1.575984,-1.980026,-1.942028,1.882710,-0.069679
1,-1.199105,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.492674,-1.332226,...,1.617871,1.091116,0.846686,-1.108050,-1.060432,-1.107444,-1.186383,-1.098773,1.001093,1.458929
2,-0.724190,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.641562,-0.942479,...,0.384265,-0.675186,1.647267,-0.570105,-0.559276,-0.651569,-0.292900,-0.301154,0.712594,1.493043
3,-0.163232,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,0.819294,-0.745594,...,-0.327086,-0.315003,0.724545,-0.085751,-0.123447,-0.265127,0.301776,0.218363,-0.316961,0.383215
4,0.285380,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,1.120769,0.224771,...,-1.445844,-1.068703,-0.075352,0.346036,0.257438,0.373867,0.761640,0.571261,-0.618515,-0.656508
5,0.639974,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.746767,0.894033,...,-0.861321,-1.037776,-0.434222,0.726276,0.554545,0.746270,0.796478,0.594785,-0.977369,-1.225666
6,1.086002,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.491969,1.290031,...,-0.242546,-0.226057,-1.118905,1.055993,0.788918,1.132417,0.771469,0.600128,-0.629670,-1.239651
7,1.510307,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,0.499410,1.325550,...,-0.525521,0.251479,-1.611837,1.336208,1.735632,1.347570,0.827948,1.357418,-1.053882,-0.143683


train_target:


,São Paulo - Consumo de Cimento (t)
0,9148.318
1,9724.722
2,10863.790
3,11775.453
4,13476.268
5,13304.301
6,14534.980
7,14871.943


test_input:


...

,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
17,-0.733293,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-0.517117,0.377546,...,-0.416421,-0.692233,-1.206794,-0.064924,0.776026,-1.154968,-1.420169,0.021261,0.852494,2.586291


test_target:


,São Paulo - Consumo de Cimento (t)
17,12113.838


1/1 [==============================] - 0s 35ms/step
Error: 1787.6651484374997




[14882.373046875,
 14930.455078125,
 14155.751953125,
 14401.849609375,
 12230.8232421875,
 11125.2236328125,
 10856.8125,
 9910.5439453125,
 10387.0458984375,
 10326.1728515625]

In [26]:
display(targets)
display(predictions)

[15159.683,
 15089.297,
 13714.511285714289,
 11767.65342857143,
 9811.084285714289,
 9918.501,
 10001.077,
 10458.276,
 11181.809,
 12113.838]

[14882.373046875,
 14930.455078125,
 14155.751953125,
 14401.849609375,
 12230.8232421875,
 11125.2236328125,
 10856.8125,
 9910.5439453125,
 10387.0458984375,
 10326.1728515625]

In [27]:
mae = mean_absolute_error(predictions, targets)
mae

1112.3946117187493

In [28]:
porcentage = mae/np.mean(targets)
porcentage

0.09330938221984207